In [1]:
import pandas as pd
base_file_path = "../../files"

officer_prof_df = pd.read_csv(base_file_path + "/profiles/officer_profile.csv")
stops_df = pd.read_csv(base_file_path + "/events/stops_full.csv")

def normalize_sex(sex):
  if (sex == "M" or sex == "MALE"):
    return "MALE"
  elif (sex == "F" or sex == "FEMALE"):
    return "FEMALE"
  else:
    return "NA"

def normalize_race(race):
  if race in ["WHITE", "BLACK", "HISPANIC", "ASIAN/PACIFIC ISLANDER", "NATIVE AMERICAN/ALASKAN NATIVE"]:
    return race
  elif race == "WHITE HISPANIC":
    return "HISPANIC"
  elif race in ["AMER IND/ALASKAN NATIVE", "NATIVE AMERICAN"]:
    return "NATIVE AMERICAN/ALASKAN NATIVE"
  else:
    return race

join_left_cols = ["first_off_first_name", "first_off_last_name", "first_off_sex", "first_off_race"]
join_right_cols = ["off_first_name", "off_last_name", "off_sex", "off_race"]

stops_df.dropna(axis="index", how="any", subset=join_left_cols, inplace=True)
officer_prof_df.dropna(axis="index", how="any", subset=join_right_cols, inplace=True)

stops_df["first_off_sex"] = stops_df["first_off_sex"].transform(normalize_sex)
stops_df["second_off_sex"] = stops_df["second_off_sex"].transform(normalize_sex)
stops_df["first_off_race"] = stops_df["first_off_race"].transform(normalize_race)
stops_df["second_off_race"] = stops_df["second_off_race"].transform(normalize_race)
officer_prof_df["off_sex"] = officer_prof_df["off_sex"].transform(normalize_sex)
officer_prof_df["off_race"] = officer_prof_df["off_race"].transform(normalize_race)

stops_merged_df = pd.merge(stops_df, officer_prof_df, how="left", left_on=join_left_cols, right_on=join_right_cols, suffixes=("_stop", "_first_off"), validate="m:m")
join_left_cols = ["second_off_first_name", "second_off_last_name", "second_off_sex", "second_off_race"]
second_off_stops = stops_merged_df.dropna(axis="index", how="any", subset=join_left_cols)
second_off_stops_merged_df = pd.merge(second_off_stops, officer_prof_df, how="left", left_on=join_left_cols, right_on=join_right_cols, suffixes=("_stop", "_second_off"), validate="m:m")
pd.merge(stops_merged_df, second_off_stops_merged_df, how="left").to_csv(base_file_path + "/events/officer_id_merged/stops_merged_filter_1.csv")


/tmp/ipykernel_12214/3820399968.py:5: DtypeWarning: Columns (3,5,6,7,8,10,11,12,13,20,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  stops_df = pd.read_csv(base_file_path + "/events/stops_full.csv")
